# Data Processing and splitting

In [1]:
from logic.processing import load_data, preproc, balance_dataset, data_filter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.pipeline import make_pipeline
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import EarlyStopping
import gensim.downloader as api

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
2024-12-03 12:46:07.018856: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_train = load_data('drugsComTrain_raw.csv')
df_test = load_data('drugsComTest_raw.csv')

In [3]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
df_train_filter = data_filter(df_train)
df_test_filter = data_filter(df_test)

In [5]:
df_test_filter.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment,review_length
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22,1,68
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17,1,48
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35,1,143
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4,1,149
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on...",9,21-Apr-09,32,1,64


In [6]:
df_train_prep = preproc(df_train_filter)
df_test_prep = preproc(df_test_filter)

In [7]:
X_train = df_train_prep['clean']
y_train = df_train_prep['sentiment']

In [8]:
X_test = df_test_prep['clean']
y_test = df_test_prep['sentiment']

In [9]:
X_train_b,y_train_b = balance_dataset(X_train,y_train)
X_test_b,y_test_b = balance_dataset(X_test,y_test)

In [10]:
len(y_train_b[y_train_b==0])==len(y_train_b[y_train_b==1])

True

# Tokenize

In [11]:
X_train_tk = [text_to_word_sequence(_) for _ in X_train_b]

In [12]:
X_test_tk = [text_to_word_sequence(_) for _ in X_test_b]

# Embedding

In [13]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

In [14]:
print(len(word2vec_transfer.key_to_index))
print(len(word2vec_transfer['dog']))

400000
50


In [17]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [18]:
X_train_embed = embedding(word2vec_transfer, X_train_tk)
X_test_embed = embedding(word2vec_transfer, X_test_tk)

# Padding

In [19]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [20]:
X_train_pad.shape,X_test_pad.shape

((56314, 200, 50), (19092, 200, 50))

# Model Building

In [22]:
def init_model():
    model = Sequential()
    model.add(InputLayer((200,50)))
    model.add(layers.Masking(mask_value=0.))
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [23]:
model = init_model()

In [24]:
model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 200, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20)             │         5,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           315 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,011 (23.48 KB)

 Trainable params: 6,011 (23.48 KB)

 Non-trainable params: 0 (0.00 B)

# Model training

In [26]:
es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train_pad, y_train_b, 
          batch_size = 64,
          epochs=15,
          validation_split=0.2,
          callbacks=[es]
         )

Epoch 1/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 69s 88ms/step - accuracy: 0.6128 - loss: 0.6504 - val_accuracy: 0.6863 - val_loss: 0.5890
Epoch 2/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 74s 106ms/step - accuracy: 0.7078 - loss: 0.5666 - val_accuracy: 0.7134 - val_loss: 0.5653
Epoch 3/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 76s 107ms/step - accuracy: 0.7366 - loss: 0.5302 - val_accuracy: 0.7559 - val_loss: 0.4976
Epoch 4/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 65s 84ms/step - accuracy: 0.7603 - loss: 0.4957 - val_accuracy: 0.7157 - val_loss: 0.5780
Epoch 5/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 86ms/step - accuracy: 0.7778 - loss: 0.4714 - val_accuracy: 0.7810 - val_loss: 0.4649
Epoch 6/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 54s 77ms/step - accuracy: 0.7894 - loss: 0.4538 - val_accuracy: 0.7921 - val_loss: 0.4436
Epoch 7/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 56s 80ms/step - accuracy: 0.8024 - loss: 0.4332 - val_accuracy: 0.7970 - val_loss: 0.4349
Epoch 8/15
704/704 ━━━━━━━━━━━━━━━━━━━━ 57s 80ms/step - accuracy: 0.8098 - loss: 0.4188 

# Evaluation

In [27]:
res = model.evaluate(X_test_pad, y_test_b, verbose=0)

In [28]:
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 83.480%


In [ ]:
y_pred = model.predict(X_test_pad)

In [ ]:
y_pred = np.round(y_pred)

In [ ]:
print(classification_report(y_test_b, y_pred))